**使用dask加速科学计算**

@author: Rui Zhu  
@data: 2024-10-07  
@follow:  
    - https://docs.dask.org/en/stable/index.html  
    - https://examples.dask.org/applications/embarrassingly-parallel.html  
@install: pip install "dask[complete]"  (安装完整版)

In [1]:
import time
import random
import pandas as pd
import numpy as np

from dask.distributed import Client

# 1. 使用dask处理Pandas表格
- https://docs.dask.org/en/stable/dataframe.html